In [ ]:
!pip install pycaret
!pip install pycaret[full]
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.4 MB/s eta 0:0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.api as tsa
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
%matplotlib inline
from matplotlib import font_manager, rc
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='NanumBarunGothic')
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pycaret.regression import *
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.dummy import DummyRegressor

#Window Size 반영한 데이터 생성

In [ ]:
df = pd.read_excel("/content/last_data_final_1.xlsx")
df_population = pd.read_excel("/content/총인구.xlsx")

df_population.set_index('월별', inplace=True)
df.set_index('월별', inplace=True)

def create_lagged_dataframe(df,df_population, window_size):
    new_df = df.copy()

    for i in range(1, window_size + 1):
        lagged_df = df.shift(i)
        lagged_df.columns = [col + f'_lag{i}' for col in df.columns]
        new_df = pd.concat([new_df, lagged_df], axis=1)

    new_df = new_df.dropna()

    new_df = new_df.drop(df.columns[df.columns != '총인구'], axis=1)

    df_population=df_population.iloc[1:]
    new_df = pd.concat([df_population, new_df], axis=1)
    new_df = new_df.dropna()

    return new_df


lagged_dfs = []

for i in range(1, 13):
    lagged_dfs.append(create_lagged_dataframe(df,df_population, i))
    exec(f'df_{i} = lagged_dfs[{i - 1}]')

df_list = []

for i in range(1, 13):
    df_name = f'df_{i}'
    df_list.append(eval(df_name))

#모델 적용
model=Lasso(random_state=2018150468)

#PYCARET

In [ ]:
def generate_pred_model(df, df_population,i, window_size, model_type):
    df_new = create_lagged_dataframe(df,df_population, window_size)
    reg = setup(data=df_new, fold_strategy='timeseries', fold=5, target='총인구', session_id=2018150468,data_split_shuffle=False, fold_shuffle=False)

    model = create_model(model_type)
    tun_model = tune_model(model, n_iter=5)
    final_model = finalize_model(tun_model)

    cv_results = pull()
    avg_rmse = cv_results['RMSE'].mean()
    avg_mae = cv_results['MAE'].mean()

    result_df = pd.DataFrame({'Model_Type': model_type, 'Average_RMSE': avg_rmse, 'Average_MAE': avg_mae}, index=[i])

    return result_df, final_model

In [ ]:
result_lasso,lasso_final=generate_pred_model(df,df_population,1,1,'lasso')

,Description,Value
0,Session id,2018150468
1,Target,총인구
2,Target type,Regression
3,Original data shape,"(111, 44)"
4,Transformed data shape,"(111, 44)"
5,Transformed train set shape,"(77, 44)"
6,Transformed test set shape,"(34, 44)"
7,Numeric features,43
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4766.8802,35347465.0016,5945.3734,0.8483,0.0092,0.0074
1,18372.8229,458628393.0410,21415.6110,-1.2860,0.0302,0.0266
2,6323.5156,56483086.2503,7515.5230,0.8788,0.0099,0.0084
3,7062.6771,69383015.1094,8329.6468,0.8185,0.0102,0.0086
4,11427.1198,213111681.1481,14598.3452,-0.1514,0.0168,0.0130
Mean,9590.6031,166590728.1101,11560.8999,0.2217,0.0153,0.0128
Std,4915.9486,158901241.7724,5739.0176,0.8477,0.0079,0.0071


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2398.8229,8729334.3320,2954.5447,0.9625,0.0046,0.0038
1,4352.8646,30402739.0352,5513.8679,0.8485,0.0078,0.0063
2,5859.4375,48754383.6725,6982.4339,0.8954,0.0092,0.0078
3,9615.3646,112181661.6523,10591.5845,0.7066,0.0130,0.0118
4,10312.2760,150342393.4808,12261.4189,0.1877,0.0141,0.0118
Mean,6507.7531,70082102.4346,7660.7700,0.7201,0.0098,0.0083
Std,3035.7473,52929335.5832,3375.6045,0.2791,0.0034,0.0031


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 5 candidates, totalling 25 fits


#해석을 위한 데이터 가공

In [ ]:
coefficients = lasso_final.coef_
df_1.drop('총인구',axis=1, inplace=True)
df_1_name=df_1.columns.tolist()
coefficients_df = pd.DataFrame({'Variable': df_1_name, 'Coefficient': coefficients})

#계수가 0인 것 삭제
remove_df = coefficients_df[coefficients_df['Coefficient'] == 0]

remain_df = coefficients_df[coefficients_df['Coefficient'] != 0]


variable_list=remain_df['Variable'].tolist()

grouped_variables = {}

for variable in variable_list:

    parts = variable.split('_lag')
    if len(parts) == 2:
        base_name, lag_suffix = parts

        if lag_suffix.isdigit():
            base_name = f"{base_name}_lag"
            if base_name not in grouped_variables:
                grouped_variables[base_name] = []
            grouped_variables[base_name].append(variable)


grouped_lists = list(grouped_variables.values())


for group in grouped_lists:
    print(group)


grouped_dataframes = {}

for idx, group in enumerate(grouped_lists):

    group_dataframe = remain_df[remain_df['Variable'].isin(group)]


    group_name = f"group{idx + 1}"


    grouped_dataframes[group_name] = group_dataframe

group_names = list(grouped_dataframes.keys())



combined_dfs = []

for group_number in range(1, 24):
    group_name = f'group{group_number}'
    group_df = grouped_dataframes[group_name].reset_index(drop=True)
    group_df.columns = [f'variable{group_number}', f'coefficient{group_number}']
    combined_dfs.append(group_df)

combined_df = pd.concat(combined_dfs, axis=1)

combined_df.to_csv('combined_df_1.csv', index=False)

['외국인_lag1']
['등록외국인_lag1']
['외국국적동포_lag1']
['지가지수_lag1']
['지가변동률_수도권_lag1']
['매매가격지수_아파트_수도권_lag1']
['매매가격지수_아파트_서울_lag1']
['매매가격지수_아파트_화성시_lag1']
['전세가격지수_아파트_수도권_lag1']
['전세가격지수_아파트_서울_lag1']
['전세가격지수_아파트_화성시_lag1']
['총전입_lag1']
['총전출_lag1']
['PI_P_lag1']
['PI_C_lag1']
['PI_SI_lag1']
['PI_PA_lag1']
['CD_lag1']
['CP_lag1']
['BaseRate_lag1']
['실업률_lag1']
['CPI_F_lag1']
['CPI_C_lag1']
['CPI_H_lag1']
['CPI_HE_lag1']
['CPI_T_lag1']
['CPI_E_lag1']
['CPI_M_lag1']
['도시지역_주거지역_lag1']
['도시지역_상업지역_lag1']
['관리지역_lag1']
['농림지역_lag1']
['의료인력_총합_lag1']
['자동차_총합_lag1']
['주택 거래 현황_lag1']
['지가변동률 _농촌생활권_lag1']
['지가변동률_도시생활이용권_lag1']
['0세_19세 총인구_lag1']
['20세_39세 총인구_lag1']
['40세_59세 총인구_lag1']
['60세 이상_총인구_lag1']
